In [6]:
import urllib.request, urllib.parse, urllib.error
from urllib.request import urlopen
from bs4 import BeautifulSoup
import ssl
import re

In [7]:
cntxt = ssl.create_default_context()
cntxt.check_hostname = False
cntxt.verify_mode = ssl.CERT_NONE

In [8]:

url = input('Please enter the website to obtain data from: ')
if len(url) < 1: url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

Please enter the website to obtain data from:  https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M 


In [9]:

#Use a file-handle like object to open the url
html = urlopen(url, context= cntxt).read() #Read slurps everything in #Note that is additional function written at end

#Use BeautifulSoup to parse
soup = BeautifulSoup(html, 'html.parser')

In [10]:
type(soup)

bs4.BeautifulSoup

In [11]:
#Try retrieving 'tr' tags
#print(soup)
tags = soup('tr')
print('Total tags extracted: ', len(tags),'\n')

#Look at the tag extracted
count = 0 #Initialise counter to count iterations & if rqd, help break out of loop
dict = {} #Initialise empty dictionary to store postcode as KEY, boroughs and neighbourhoods as VALUES
list = [] #Initialise empty list to hold the borough + neighbourhood info

#Loop through the tags
for i in tags:
    count = count + 1 #Increase counter value beginning through each iteration
    if count == 290: break #For limiting output and stopping the loop from running + length of tags for some reason does not correspond to actual number of elements
    
    #Skip if Borough is Not Assigned
    if i.contents[3].text == 'Not assigned':
        continue
        
    #Skip if text is Postcode
    if i.contents[1].text == 'Postcode':
        continue
    
    #If pincode already exists, eg M5A, M6A, append new data
    if i.contents[1].text in dict: 
        
        # Append the new data to the existing array at this slot with the following NEW SYNTAX
        #dict[existing_key].append(value)
        dict[i.contents[1].text].append(i.contents[5].text.rstrip())
        
    #If borough exists but not the neighbourhood, run the following    
    elif re.search('[a-z]', i.contents[3].text) and re.search('No.*', i.contents[5].text):
        list.append(i.contents[3].text), list.append(i.contents[3].text)
        dict[i.contents[1].text] = list
    else:
        list.append(i.contents[3].text)
        list.append(i.contents[5].text.rstrip())#rstrip gets rid of newline char.
        dict[i.contents[1].text] = list
    
    #Would need to reset the list to empty after each round otherwise each successive iteration will bloat up the key-value
    list = []
    

#print(dict)

print('\nTotal post-codes with borough info:', len(dict))

Total tags extracted:  294 


Total post-codes with borough info: 103


In [12]:
import pandas as pd

In [13]:
data = pd.DataFrame.from_dict(dict, orient= 'index')

In [14]:
data.head()

,0,1,2,3,4,5,6,7,8
M3A,North York,Parkwoods,None,None,None,None,None,None,None
M4A,North York,Victoria Village,None,None,None,None,None,None,None
M5A,Downtown Toronto,Harbourfront,Regent Park,None,None,None,None,None,None
M6A,North York,Lawrence Heights,Lawrence Manor,None,None,None,None,None,None
M7A,Queen's Park,Queen's Park,None,None,None,None,None,None,None


In [15]:
data.index.name = 'PostalCode'
data.head()

,0,1,2,3,4,5,6,7,8
PostalCode,,,,,,,,,
M3A,North York,Parkwoods,None,None,None,None,None,None,None
M4A,North York,Victoria Village,None,None,None,None,None,None,None
M5A,Downtown Toronto,Harbourfront,Regent Park,None,None,None,None,None,None
M6A,North York,Lawrence Heights,Lawrence Manor,None,None,None,None,None,None
M7A,Queen's Park,Queen's Park,None,None,None,None,None,None,None


In [16]:
data.reset_index(inplace= True)
data.head()

,PostalCode,0,1,2,3,4,5,6,7,8
0,M3A,North York,Parkwoods,None,None,None,None,None,None,None
1,M4A,North York,Victoria Village,None,None,None,None,None,None,None
2,M5A,Downtown Toronto,Harbourfront,Regent Park,None,None,None,None,None,None
3,M6A,North York,Lawrence Heights,Lawrence Manor,None,None,None,None,None,None
4,M7A,Queen's Park,Queen's Park,None,None,None,None,None,None,None


In [17]:
data.columns.values[1] = 'Borough'#We need to drill down to the array, accessed with .values
data.head()

,PostalCode,Borough,1,2,3,4,5,6,7,8
0,M3A,North York,Parkwoods,None,None,None,None,None,None,None
1,M4A,North York,Victoria Village,None,None,None,None,None,None,None
2,M5A,Downtown Toronto,Harbourfront,Regent Park,None,None,None,None,None,None
3,M6A,North York,Lawrence Heights,Lawrence Manor,None,None,None,None,None,None
4,M7A,Queen's Park,Queen's Park,None,None,None,None,None,None,None


In [18]:
data.columns = data.columns.tolist()
data[['Borough']].head()

,Borough
0,North York
1,North York
2,Downtown Toronto
3,North York
4,Queen's Park
